# ETL Project
----

### Yelp Web Scraping

In this notebook are included the following tasks:

* __Extract__

   Search the `Coffee and Tea` businesses near Mexico City area in Yelp.com and using web scraping, got business name and location from each HTML page.


* __Transform__

   Using Python, Splinter in Chrome, and BeautifulSoup got 1,000 business.


* __Load__ 
   
   The dataset was stored permanently in a Mongo cluster.

---


In [1]:
# Depemdencies
from splinter import Browser
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [2]:
# Splinter for Chrome
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URL of page to be scraped
url = 'https://www.yelp.com/search?find_desc=Coffee+%26+Tea&find_loc=Mexico+City%2C+D.F.%2C+Mexico&ns=1'
browser.visit(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

In [6]:
# Get the number of results
total = soup.find_all('p', class_='lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--right__373c0__3ARv7')         
for x in total:
    number = x.text.strip()
    print(number)

Showing 1-30 of 1221


In [7]:
# Calculate the number of pages we have to visit
pages = round(int(number.split('-')[1].split('of')[1]) / int(number.split('-')[1].split('of')[0]),0)
pages

41.0

In [17]:
# Get the information from each page 
next_link_xpath_p1 = '//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[10]/a/div/span'
next_link_xpath_ps = '//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[11]/a/div/span'

unit_list = []

for x in range(int(pages)):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Save the HTML page to a file for later verification
    file_name = "../Yelp/page" + str(x + 1) + ".html"
    with open(file_name, "w") as f:
        f.write(str(soup.encode("utf-8")))

    # Retrieve all li page elements
    elements = soup.find_all('li', class_='lemon--li__373c0__1r9wz border-color--default__373c0__2oFDT')
    for element in elements:
        h3 = element.find('h3')
        address = element.find('address')
        div = element.find('div', class_='lemon--div__373c0__1mboc u-space-b1 border-color--default__373c0__2oFDT')
        if h3 !=None and address != None and div != None: 
            unit = str(h3.text.strip() + ";" + str(address.text.strip()) + ";" + str(div.text.strip()) + "\n" )
            if unit not in unit_list:
                unit_list.append(unit)
    
    # Click the 'Next' button on each page by xpath to avoid exceptions in Chrome
    try:
        # Code that might cause an exception
        if x == 0:
            browser.find_by_xpath(next_link_xpath_p1)[0].click() 
        else:
            browser.find_by_xpath(next_link_xpath_ps)[0].click() 

    except Exception as e:
        # Code to be ejecuted in case an exception arises
        print("Exception - Page "+ str(x + 1) + " - " + str(e) + "\n")

print("Scraping Complete\n")


Exception - Page 34 - no elements could be found with xpath "//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[11]/a/div/span"

Exception - Page 35 - no elements could be found with xpath "//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[11]/a/div/span"

Exception - Page 36 - no elements could be found with xpath "//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[11]/a/div/span"

Exception - Page 37 - no elements could be found with xpath "//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[11]/a/div/span"

Exception - Page 38 - no elements could be found with xpath "//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[11]/a/div/span"

Exception - Page 39 - no elements could be found with xpath "//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/div[1]/div/div[2]/div/div[11]/a/div/span"

Exception - Page 40 - 

In [18]:
print(len(unit_list))

973


In [23]:
for unit in unit_list:
    print(unit)

1. Chiquitito Café;Alfonso Reyes 232;Condesa

2. The Black Rabbit;Xola 1603;Narvarte

3. Cardinal;Calle Córdoba 132;Roma Norte

4. Solemnus Café;Juan Sánchez Azcona 541;Del Valle

5. Enhorabuena Café;Calle Atlixco 13;Condesa

6. Tomás;Tamaulipas 66;Condesa

7. Panadería Rosetta;Colima 179;Roma Norte

8. Café Avellaneda;Higuera 40;Coyoacán

9. Buna;Orizaba 42;Roma Norte

10. Blend Station;Av. Tamaulipas 60;Condesa

11. Salem Witch Store & Coffee;Diagonal San Antonio 1747;Narvarte

12. Cucurucho;Chiapas 183;Roma Norte

13. Turco’s Coffee;Diagonal San Antonio 1810;Narvarte

14. La Ventanita;Plaza Villa De Madrid 13;Condesa

15. Dosis Café;Av. Álvaro Obregón 24;Roma Norte

16. Café Monteabuelo;Calle Andalucía 148;Álamos

17. Don Porfirio;Av. Juárez 14;Centro Poniente

18. Cardinal;Campeche 346;Condesa

19. Café Do Brasil “La Balsa”;Dr. José María Vértiz 822;Narvarte

20. Amsterdam Coffee Shop;Filadelfia 102-B;Nápoles

21. El Moro;Frontera 122;Roma Norte

22. Churrería El Moro;Eje Central L


851. Cassava Roots;Av. Vasco de Quiroga 3815;Santa Fe

852. Truffy;Av. San Fernando 649 Loc. 46;Tlalpan

853. Le Pain Quotidien;Calle Nueva York 248;Nápoles

854. Cafetería Zoe´s;Av. Santa Úrsula 233;

855. Café el silo;Hamburgo 268;Juárez

856. Los Bisquets Obregón;Canal de Miramontes 3280;Granjas Coapa

857. Pixan Café;Calle Lisboa 51;Juárez

859. Nuestro Cafe Illy;Miguel Laurent;Santa Cruz Atoyac

860. Café B;Dr. Mora 9;Centro Poniente

861. Delirio;Monterrey 116 B;Roma Norte

863. Café Río Elba;Río Elba 31;Cuauhtémoc

864. La Galería;Gobernador José María Tornel 60;San Miguel Chapultepec

865. Cup Stop;Marina Nacional 154 Local 4;

866. Starbucks;Calz de Las Aguilas 1953;

867. Finca Santa Veracruz;Av. México Coyoacán 389;General Anaya

868. Pethra Café y Crepas;Calz. del Hueso 503;Granjas Coapa

869. V. Café;Terminal 1;

870. Maru;Eugenia 122 Loc.1;Del Valle

871. Cafe Central;Campos Eliseos 400;Polanco

872. Central Café;Guanabana 154;

873. Ojo de Agua;Milán 44;Juárez

874. Sta

In [25]:
# Initialize PyMongo to work with MongoDBs

# Caution. The following line was modified to avoid publishing the password in GitHub, please see note in the Boot Camp Spot
_mongoClusterURI = ''

dbClient = MongoClient(_mongoClusterURI)
db = dbClient["etl_project"]

# Get the information from the list and store it in mongo
for unit in unit_list:
    name, street, neigh = unit.split(";")
    db.cafesyelp.insert_one({'name': name.replace('xa0',' ').strip(), 'street': street.strip(), 'neighborhood': neigh.strip()})

print("The insertions in the database are finished")

The insertions in the database are finished


In [30]:
# Close the browser after the scraping
browser.quit()

# Notes

At this point, we had __three problems__ to solve:

- Yelp showed that the search results were equal to 1,221 records, however after the record 990 the `Next` link was not working properly, it showed a message that Yelp could not find more results. After trying manual requests to retrieve the records from 1,001 to 1,221, without success, a decision to keep only 1,000 records was made.

     See [yelp_error](yelp_error.png)
     

- Even though the web scraping worked fine until the 990th record, only 973 records could be retrieved and inserted in Mongo. The records that could not be inserted in the first try, were retrieved directly from the previously saved HTML page  and then inserted in Mongo. Also, a manual request was made to retrieve from record 991 to 1,000 and to insert them in the database. See the following cells.


- Some records did not have the neighborhood or it was in the field `street`, therefore was necessary to make some queries  in Mongo, these are showed in the Colaboratory Jupiter Notebook.

In [3]:
# Insert the records that was not possible to obtain in the first web scraping using the htmls saved for verification
missing_list = ['92. Flora Café;Park Plaza;Santa Fé',
                '139. Amado Pasteleria;Hyatt Regency;Polanco',
                '385. DotCom Café;Aeropuerto Internacional de la Ciudad de México;Peñón de los Baños',
                '506. Cielito Café;Miyana;Granada',
                '525. Starbucks;Lomas Plaza;Las Lomas',
                '572. Starbucks;Aeropuerto Internacional de la Ciudad de México;Peñón de los Baños',
                '614. Le Pain Quotidien;Plaza Las Aguilas;Las Aguilas',
                '628. Red Tea Camelia;Mundo E;Tlalnepantla de Baz',
                '666. Starbucks;Plaza Centro Coyoacán;General Anaya',
                '744. Maison Kayser;Aeropuerto Internacional de la Ciudad de México. Terminal 2;Peñón de los Baños',
                '758. Galeria de Café;Emporio Reforma;Juárez',
                '817. Starbucks;Antara;Granada',
                '818. Finca Kafeto;Garden Santa Fé;Santa Fé',
                '858. Maison Kayser;Aeropuerto Internacional de la Ciudad de México;Peñón de los Baños',
                '862. Giornale;Pabellón Metepec;Metepec, México',
                '954. Maison Kayser;Paseo Arcos Bosques;Santa Fé',
                '975. The Italian Coffee Company;Plaza Laurel;Cuernavaca, Morelos',
                '991. Starbucks;Florencia 266;Juárez',
                '992. Starbucks;Salazar 40;Cuernavaca, Morelos',
                '993. Café Chic;Ermita Iztapalapa 2349;Iztapalapa',
                '994. Churreria del Carmen;Plaza del Carmen 4;San Ángel',
                '995. London Fresh;Av. Libertador Bernado O´Higgins 510;Santiago Tianguistenco, México',
                '996. Los Arcos;José María Iglesias 26;Tabacalera',
                '997. Cafetzalli;Av. Miguel Ángel de Quevedo 616-A;Coyoacán',
                '998. Starbucks;Agrarismo 208;Escandón',
                '999. 7 Eleven;Av. México S/N;Naucalpan, México',
                '1000. Wings;Calz México Tacuba 94;Anáhuac Sección I'
            ]
missing_list

['92. Flora Café;Park Plaza;Santa Fé',
 '139. Amado Pasteleria;Hyatt Regency;Polanco',
 '385. DotCom Café;Aeropuerto Internacional de la Ciudad de México;Peñón de los Baños',
 '506. Cielito Café;Miyana;Granada',
 '525. Starbucks;Lomas Plaza;Las Lomas',
 '572. Starbucks;Aeropuerto Internacional de la Ciudad de México;Peñón de los Baños',
 '614. Le Pain Quotidien;Plaza Las Aguilas;Las Aguilas',
 '628. Red Tea Camelia;Mundo E;Tlalnepantla de Baz',
 '666. Starbucks;Plaza Centro Coyoacán;General Anaya',
 '744. Maison Kayser;Aeropuerto Internacional de la Ciudad de México. Terminal 2;Peñón de los Baños',
 '758. Galeria de Café;Emporio Reforma;Juárez',
 '817. Starbucks;Antara;Granada',
 '818. Finca Kafeto;Garden Santa Fé;Santa Fé',
 '858. Maison Kayser;Aeropuerto Internacional de la Ciudad de México;Peñón de los Baños',
 '862. Giornale;Pabellón Metepec;Metepec, México',
 '954. Maison Kayser;Paseo Arcos Bosques;Santa Fé',
 '975. The Italian Coffee Company;Plaza Laurel;Cuernavaca, Morelos',
 '9

In [4]:
# Initialize PyMongo to work with MongoDBs

# Caution. The following line was modified to avoid publishing the password in GitHub, please see note in the Boot Camp Spot
_mongoClusterURI = ''

dbClient = MongoClient(_mongoClusterURI)
db = dbClient["etl_project"]

# Get the information from the list and store it in mongo
for miss in missing_list:
    name, street, neigh = miss.split(";")
    db.cafesyelp.insert_one({'name': name.strip(), 'street': street.strip(), 'neighborhood': neigh.strip()})

print("The rest of the insertions in the database are finished")

The rest of the insertions in the database are finished
